In [1]:
import tkinter as tk
import tkinter.ttk as ttk

import re
import pandas as pd
import numpy as np

import ipynb_importer
from publish_subscribe import Reader
from publish_subscribe import get_poems
from tkinter import *
from tkinter import (Tk, Text, Scrollbar)
from tkinter.constants import (HORIZONTAL, VERTICAL, RIGHT, LEFT, X, Y, BOTH, BOTTOM, YES)
#from doc_TF_IDF import query_word, search1, search2, search3


importing Jupyter notebook from publish_subscribe.ipynb


In [7]:
target_list = pd.read_csv('target_list.csv').iloc[:,1].values

array(['䌽', '一', '一一', ..., '龙', '龙门', '龟'], dtype=object)

In [ ]:
MAX_POEM_SHOWN = 20
def search1(weight, query,similar_dict):
    l = weight.loc[query]
    doc = l[l==l].sort_values(ascending = False)
    res = list(doc.index.values)
    num = len(res)
    print(num)
    #如果符合条件的诗歌太少，进行近义词搜索
    if len(res) < MAX_POEM_SHOWN:
        similar_d = similar_dict[query]
        i = 0
        while num < MAX_POEM_SHOWN:
            cur_key = similar_d[i][0]
            print(cur_key)
            l = weight.loc[cur_key]
            doc = l[l==l].sort_values(ascending = False)
            res.extend(list(doc.index.values))
            i+=1
            num += len(doc)
    return res[:min(20,len(res))]

def search2(weight, query_list,similar_dict):    
    word1 = query_list[0]
    word2 = query_list[1]
    l1 = weight.loc[word1]
    doc1 = pd.DataFrame(l1[l1==l1]).reset_index()
    l2 = weight.loc[word2]
    doc2 = pd.DataFrame(l2[l2==l2]).reset_index()

    #取交集
    alpha = 0.6
    intersection = pd.merge(doc1, doc2, on = 'index', how = 'inner')
    intersection['weight'] = intersection[word1] * alpha + intersection[word2] * (2-alpha)
    intersection = intersection.loc[:,['index', 'weight']]
    concat_list = []
    if len(intersection)>0:
        concat_list.append(intersection)

    if len(intersection) < MAX_POEM_SHOWN:
        if word1 in target_list:
            similar1 = similar_dict[word1][0][0]
            similar1_cor = similar_dict[word1][0][1]
            l1 = weight.loc[similar1]
            doc11 = pd.DataFrame(l1[l1==l1]).reset_index()
            intersection1 = pd.merge(doc11, doc2, on = 'index', how = 'inner')
            intersection1['weight'] = intersection1[similar1] * similar1_cor * alpha + intersection1[word2] * (2-alpha)
            intersection1 = intersection1.loc[:,['index', 'weight']]
            if len(intersection)>0:
                concat_list.append(intersection1)

        if word2 in target_list:
            similar2 = similar_dict[word2][0][0]
            similar2_cor = similar_dict[word2][0][1]
            l2 = weight.loc[similar2]
            doc21 = pd.DataFrame(l2[l2==l2]).reset_index()
            intersection2 = pd.merge(doc1, doc21, on = 'index', how = 'inner')
            intersection2['weight'] = intersection2[word1] *  alpha + intersection2[similar2] * similar2_cor *(2-alpha)
            intersection2 = intersection2.loc[:,['index', 'weight']]
            if len(intersection)>0:
                concat_list.append(intersection2)


    union = pd.merge(doc1, doc2, on = 'index', how = 'outer')
    left = union.append(intersection).drop_duplicates(keep = False)
    left['weight'] = left.apply(lambda x: x.iloc[1] if x.iloc[1] == x.iloc[1] else x.iloc[2], axis = 1)
    left = left.loc[:,['index','weight']]
    concat_list.append(left)
    concat = pd.concat(concat_list).drop_duplicates('index')
    concat = concat.sort_values('weight', ascending = False)
    return concat['index'].values[:min(len(concat),20)]

def search3(weight, query_list,similar_dict):
    word1 = query_list[0]
    word2 = query_list[1]
    word3 = query_list[2]
    l1 = weight.loc[word1]
    doc1 = pd.DataFrame(l1[l1==l1]).reset_index()
    l2 = weight.loc[word2]
    doc2 = pd.DataFrame(l2[l2==l2]).reset_index()
    l3 = weight.loc[word3]
    doc3 = pd.DataFrame(l3[l3==l3]).reset_index()

    #取三个关键词的交集
    intersection1 = pd.merge(doc1, doc2, on = 'index', how = 'inner')
    intersection2 = pd.merge(doc1, doc3, on = 'index', how = 'inner')
    intersection3 = pd.merge(doc2, doc3, on = 'index', how = 'inner')

    intersection = pd.merge(intersection1, doc3, on = 'index', how = 'inner')
    concat_list = []
    alpha = 1.1
    beta=1
    intersection['weight'] = intersection[word1] * alpha + intersection[word2] * beta + intersection[word3] * (3-alpha-beta)
    concat_list = []
    intersection=intersection.loc[:,['index','weight']]
    concat_list.append(intersection)
    
    #只包含两个关键词的集合
    left1 = intersection1.append(intersection).drop_duplicates(keep = False)
    left1['weight'] = left1.apply(lambda x: alpha*x.iloc[1]+(2-alpha)*x.iloc[2] if x.weight != x.weight else x.iloc[3], axis = 1)
    left1 = left1.loc[:,['index','weight']]

    left2 = intersection2.append(intersection).drop_duplicates(keep = False)
    left2['weight'] = left2.apply(lambda x: alpha*x.iloc[1]+(2-alpha)*x.iloc[2] if x.weight != x.weight else x.iloc[3], axis = 1)
    left2 = left2.loc[:,['index','weight']]

    left3 = intersection3.append(intersection).drop_duplicates(keep = False)
    left3['weight'] = left3.apply(lambda x: alpha*x.iloc[1]+(2-alpha)*x.iloc[2] if x.weight != x.weight else x.iloc[3], axis = 1)
    left3 = left3.loc[:,['index','weight']]
    
    #关键词替换：只替换一个
    if len(intersection) < MAX_POEM_SHOWN:
        if len(intersection1) >0 and word3 in target_list:
            similar3 = similar_dict[word3][0][0]
            similar3_cor = similar_dict[word3][0][1]
            l31 = weight.loc[similar3]
            doc31 = pd.DataFrame(l31[l31==l31]).reset_index()
            intersection31 = pd.merge(intersection1, doc31, on = 'index', how = 'inner')
            intersection31['weight'] = intersection31[word1] * alpha+intersection31[word2]* beta +intersection31[similar3]* (3-alpha-beta)*similar3_cor
            intersection31=intersection31.loc[:,['index','weight']]
            concat_list.append(intersection31)
        if len(intersection2) >0 and word2 in target_list:
            similar2 = similar_dict[word2][0][0]
            similar2_cor = similar_dict[word2][0][1]
            l21 = weight.loc[similar2]
            doc21 = pd.DataFrame(l21[l21==l21]).reset_index()
            intersection21 = pd.merge(intersection2, doc21, on = 'index', how = 'inner')
            intersection21['weight'] = intersection21[word1]*alpha+intersection21[similar2]*beta*similar2_cor +intersection21[word3]*(3-alpha-beta)
            intersection21=intersection21.loc[:,['index','weight']]
            concat_list.append(intersection21)
        if len(intersection3) >0 and word1 in target_list:
            similar1 = similar_dict[word1][0][0]
            similar1_cor = similar_dict[word1][0][1]
            l11 = weight.loc[similar1]
            doc11 = pd.DataFrame(l11[l11==l11]).reset_index()
            intersection11 = pd.merge(intersection3, doc11, on = 'index', how = 'inner')
            intersection11['weight'] = intersection11[similar1]*alpha*similar1_cor+intersection11[word2]* beta +intersection11[word3]* (3-alpha-beta)
            intersection11=intersection11.loc[:,['index','weight']]
            concat_list.append(intersection11)
    concat_list.extend([left1,left2,left3])
    res = pd.concat(concat_list).sort_values('weight', ascending = False)
    return res['index'].values[:min(len(res),20)]

def query_word(weight, query_list,similar_dict):
    
    if len(query_list) == 1:
        ret = search1(weight, query_list[0],similar_dict)
    elif len(query_list) == 2:
        ret = search2(weight, query_list,similar_dict)
    elif len(query_list) == 3:
        ret = search3(weight, query_list,similar_dict)
    else:
        ret = 'too many words'
    return ret

In [3]:
table = pd.read_excel('poem_v2.xlsx')
def get_author_list():
    author = table[table.line_number == -1]
    author.loc[:,'simple'] = author['simple'].apply(lambda x:x[2:])
    author_poem_list = author.groupby('simple').apply(lambda x:x['Poem_id'].values)
    return author_poem_list

def gather(x):
    res = []
    res.append(x[x.line_number==-100]['simple'].values[0][2:])
    res.append(x[x.line_number==-1]['simple'].values[0][2:])
    res.extend(list(x[(x.line_number!=-100)&(x.line_number!=-1)]['simple'].values))
    return res


In [ ]:
MAX_POEM_SHOWN = 20

In [4]:
class MainPanel:
    def __init__(self, reader, publisher, author_list):
        self.reader = reader
        self.publisher = publisher
        self.all_authors = author_list
        self.cur_author = ""
        poem_dist = get_poems()
        self.poem_dist = {}
        self.weight_mat = pd.read_csv('weight1.csv', index_col = 'word', engine = 'c')#分词的排序文档矩阵
        self.author_poem_list = get_author_list() #由诗人名字索引他的所有诗的id
        self.id_poems = pd.DataFrame(table.groupby('Poem_id').apply(gather)) #由诗的id索引诗的内容
        self.subscribe_dict = {} #记录已经订阅的查询结果
        self.similar_dict = np.load('similar_dict.npy',allow_pickle=True).item()
        
        for author in self.all_authors:
            self.poem_dist[author] = poem_dist[author]
    
    def start(self):
        """
        显示主界面
        """
        self.root = tk.Tk()
        self.root.geometry('400x400')
        self.root.title("古诗订阅")
        self.root.iconbitmap('cat.ico')
        self.label = tk.Label(self.root, text="我的订阅", font=(None,12)).pack(pady=20)
        self.add_button = tk.Button(self.root, text="新增订阅", font=(None,12), command=self.add_subscription).pack(pady=40)
        self.root.mainloop()
        
    def add_subscription(self):
        """
        用户点击"新增订阅"按钮，显示该界面
        """
        self.add_panel = tk.Tk()
        self.add_panel.geometry('400x300')
        self.add_panel.iconbitmap('cat.ico')
        
        label = tk.Label(self.add_panel, text="作者", font=(None, 10)).pack(pady=20)
        self.new_author_entry = tk.Entry(self.add_panel)
        self.new_author_entry.pack()
        
        label1 = tk.Label(self.add_panel, text="关键字", font=(None, 10)).pack(pady=20)
        self.new_key_entry = tk.Entry(self.add_panel)
        self.new_key_entry.pack()
        
        confirm_button = tk.Button(self.add_panel, text="确认", font=(None, 10), command=self.search_poem).pack(pady=20)
        self.hint_label = tk.Label(self.add_panel, text="", font=(None, 10))
        self.hint_label.pack()
    
    def search_poem(self):
        """
        用户点击"确认"按钮后，检查输入是否合法
        
        """
        new_author = self.new_author_entry.get()
        new_key = self.new_key_entry.get()
        substribed = False
        result = []
        if new_author != '':
            if new_author not in self.all_authors:
                self.hint_label.config(text=f"非法作者名！")
            else:
                author_search_list = list(self.author_poem_list.loc[new_author])
                result  = author_search_list
                substribed = True
                
        if new_key != '':
            query_list = new_key.split(' ')
            if len(query_list) > 3:
                self.hint_label.config(text=f"最多三个关键词！")
            else:
                key_search_list = query_word(self.weight_mat, query_list,self.similar_dict)
                key_search_list = list(map(int, key_search_list))
                if len(result) > 0:
                    result = [x for x in key_search_list if x in author_search_list]
                else:
                    result = key_search_list
                substribed = True
                
        if substribed:
            self.subscribe_dict[new_author + ' ' + new_key] = result
            print(result)
            tmp_tk = tk.Tk()
            tmp_tk.geometry("300x200")
            tk.Label(tmp_tk, text="订阅成功", font=(None, 20)).pack(pady=20)
            tk.Button(tmp_tk, text="确认", font=(None, 12), command=tmp_tk.destroy).pack(pady=20)
        else:
            tmp_tk = tk.Tk()
            tmp_tk.geometry("300x200")
            tk.Label(tmp_tk, text="搜寻结果不存在", font=(None, 20)).pack(pady=20)
            tk.Button(tmp_tk, text="确认", font=(None, 12), command=tmp_tk.destroy).pack(pady=20)
        self.add_panel.destroy()
        self.update_main_panel()
            
    def update_main_panel(self):
        """
        用户新增订阅后，更新主界面
        """
        for widget in self.root.winfo_children():
            widget.destroy()
        self.label = tk.Label(self.root, text="我的订阅", font=(None,12)).pack(pady=20)
        for subscribtion in self.subscribe_dict.keys():
            button = tk.Button(self.root, text=subscribtion, width=13, font=(None,12))
            button.bind("<Button-1>", self.show_result)
            button.pack(pady=10)
            
        self.add_button = tk.Button(self.root, text="新增订阅", font=(None,12), command=self.add_subscription).pack(pady=40)
        self.root.update()
        
    def show_result(self, event):
        """
        用户点击主界面的“订阅关键词”按钮：
        1. 显示符合关键词条件的前N首诗，一页三首
        2. 关键词高亮
        3. 可以翻页
        """
        root = tk.Tk()
        root.geometry('400x600')
        
        #1.布局界面
        cur_key = event.widget['text']

        cur_key_list = cur_key.strip().split(' ')
        print(cur_key_list)
        poem_list = self.subscribe_dict[cur_key]
        tab_main=ttk.Notebook(root)#创建分页栏
        tab_main.place(relx=0.02, rely=0.02, relwidth=0.887, relheight=0.876)
        
        poem_per_page = 4
        if len(poem_list) % poem_per_page == 0:
            page = len(poem_list) // poem_per_page
        else:
            page = len(poem_list) // poem_per_page + 1
        for j in range(min(5, page)):
            tab1=Frame(tab_main)#创建第一页框架
            tab1.place(x=100 * j,y=30)
            tab_main.add(tab1,text=' '+str(j + 1) + ' ')#将第一页插入分页栏中

            text = tk.Text(tab1, width=100, height=100)
            text.place(x=10,y=10)
            text.tag_config("tag_1", backgroun="yellow", foreground="red")# 设置 tag
            for k in range(poem_per_page):
                i = j * poem_per_page + k
                if i >= len(poem_list):
                    break
                cur_id = poem_list[i]
                cur_p = self.id_poems.loc[cur_id].values[0]
                for m in range(len(cur_p)):
                    marked = False
                    cur_sentence = cur_p[m]
                    for word in cur_key_list:
                        if word in cur_sentence:
                            #print('in')
                            start = cur_sentence.find(word)
                            end = start + len(word)
                            text.insert("insert", cur_sentence[0:start])
                            text.insert("insert",  word, "tag_1")
                            text.insert("insert", cur_sentence[end:]+'\n')
                            marked = True
                            break
                    if not marked:
                        text.insert("insert", cur_sentence+'\n')
                text.insert("insert", '\n')
        tk.Button(root, text="返回", font=(None, 12), command=root.destroy).pack(pady=20)
        
        